<a href="https://colab.research.google.com/github/asu-trans-ai-lab/AI_traffic_flow/blob/main/1A_Tic_Tac_Toe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pulp

In [15]:
import pulp

# Create a new LP problem
model = pulp.LpProblem("Tic-Tac-Toe", pulp.LpMaximize)

# Defining the variables (for simplicity, we consider only a few cells and one turn)
x_11 = pulp.LpVariable('x_11', cat='Binary')
x_12 = pulp.LpVariable('x_12', cat='Binary')
x_13 = pulp.LpVariable('x_13', cat='Binary')

# Define the current state of the board (for example, let's assume these are already filled)
# In a real scenario, these would be parameters, not variables
o_11 = 0  # Cell (1,1) is empty
o_12 = 1  # Cell (1,2) is filled by O
o_13 = 0  # Cell (1,3) is empty

# Objective Function
# For simplicity, we'll assume Player X wants to maximize their presence on the board
model += x_11 + x_12 + x_13

# Constraints
# For this example, let's add a simple constraint that only one cell can be marked by X
model += x_11 + x_12 + x_13 <= 1

# Existing state constraints (these cells are already filled by O, so X can't mark them)
model += x_12 == 0

# Solve the model
model.solve()

# Output the results
for v in model.variables():
    print(v.name, "=", v.varValue)

print("Status:", pulp.LpStatus[model.status])


x_11 = 1.0
x_12 = 0.0
x_13 = 0.0
Status: Optimal


Updated Python Code with Time Index and Opponent Variables:

In [47]:
import pulp

# Time steps
T = 5  # For example, we are looking at the 5th move

# Create a new LP problem
model = pulp.LpProblem("Tic-Tac-Toe-Time-Indexed", pulp.LpMaximize)

# Defining the variables with time index for both players X and O
x = pulp.LpVariable.dicts("x", [(i, j, t) for i in range(1, 4) for j in range(1, 4) for t in range(1, T + 1)], cat='Binary')
o = pulp.LpVariable.dicts("o", [(i, j, t) for i in range(1, 4) for j in range(1, 4) for t in range(1, T + 1)], cat='Binary')

# Example current state at t-1 (turn 4)
# Let's assume some cells are already marked by X and O
# These would typically come from the actual game state
current_state = {
    (1, 1, 1): 1, (1, 2, 1): 0, (1, 3, 1): 0,
    (2, 1, 1): 0, (2, 2, 1): 1, (2, 3, 1): 0,
    (3, 1, 1): 0, (3, 2, 1): 0, (3, 3, 1): 0,
    # ... continue for each cell and time up to t-1
}

# In each time step, each player will mark 1 cell.
# So before we apply the state, we assume first marked cell belongs to playerX, the other one belongs to playerO.

# Initialize counters for each player
x_count = 0
o_count = 0

# Applying the current state to our variables
for cell, marked in current_state.items():
    if marked:
        # If the cell is marked, assign it to the corresponding player's variable
        if x_count <= o_count:
            model += x[cell] == 1
            model += o[cell] == 0
            x_count += 1
        else:
            model += x[cell] == 0
            model += o[cell] == 1
            o_count += 1

# Constraints
# (1) Ensure each cell's occupation at time t is consistent with time t-1
for t in range(2, T + 1):  
    for i in range(1, 4):
        for j in range(1, 4):
            # If the cell is already occupied at time t-1, then it should be occupied by the same player at time t
            model += x[i, j, t] >= x[i, j, t-1]
            model += o[i, j, t] >= o[i, j, t-1]

# (2) The total amount of marked cells should be constrained
for t in range(1, T + 1):
    if t<T: # In 1st~4th turn, each player will mark 4 cells. 
        model += pulp.lpSum([x[i, j, t] for i in range(1, 4) for j in range(1, 4)]) == t
        model += pulp.lpSum([o[i, j, t] for i in range(1, 4) for j in range(1, 4)]) == t
    else:   # In 5th turn, only Player X can mark 1 cell, because only 1 cell remains.
        model += pulp.lpSum([x[i, j, t] for i in range(1, 4) for j in range(1, 4)]) == T
        model += pulp.lpSum([o[i, j, t] for i in range(1, 4) for j in range(1, 4)]) == T-1

# (3) Each cell can be only marked once
for t in range(1, T + 1):  
    for i in range(1, 4):
        for j in range(1, 4):
            model += x[i, j, t] + o[i, j, t]<=1

# Objective Function
# Assuming Player X wants to maximize their presence on the board at time T
model += pulp.lpSum([x[i, j, T] for i in range(1, 4) for j in range(1, 4)])

# Solve the model
model.solve()

# Output the results
for t in range(1, T + 1):
    print(f"Turn {t}:")
    for i in range(1, 4):
        for j in range(1, 4):
            print(f"Cell ({i}, {j}): X = {x[i, j, t].varValue}, O = {o[i, j, t].varValue}")
    print("")

print("Status:", pulp.LpStatus[model.status])


Turn 1:
Cell (1, 1): X = 1.0, O = 0.0
Cell (1, 2): X = 0.0, O = 0.0
Cell (1, 3): X = 0.0, O = 0.0
Cell (2, 1): X = 0.0, O = 0.0
Cell (2, 2): X = 0.0, O = 1.0
Cell (2, 3): X = 0.0, O = 0.0
Cell (3, 1): X = 0.0, O = 0.0
Cell (3, 2): X = 0.0, O = 0.0
Cell (3, 3): X = 0.0, O = 0.0

Turn 2:
Cell (1, 1): X = 1.0, O = 0.0
Cell (1, 2): X = 0.0, O = 0.0
Cell (1, 3): X = 1.0, O = 0.0
Cell (2, 1): X = 0.0, O = 1.0
Cell (2, 2): X = 0.0, O = 1.0
Cell (2, 3): X = 0.0, O = 0.0
Cell (3, 1): X = 0.0, O = 0.0
Cell (3, 2): X = 0.0, O = 0.0
Cell (3, 3): X = 0.0, O = 0.0

Turn 3:
Cell (1, 1): X = 1.0, O = 0.0
Cell (1, 2): X = 1.0, O = 0.0
Cell (1, 3): X = 1.0, O = 0.0
Cell (2, 1): X = 0.0, O = 1.0
Cell (2, 2): X = 0.0, O = 1.0
Cell (2, 3): X = 0.0, O = 0.0
Cell (3, 1): X = 0.0, O = 0.0
Cell (3, 2): X = 0.0, O = 0.0
Cell (3, 3): X = 0.0, O = 1.0

Turn 4:
Cell (1, 1): X = 1.0, O = 0.0
Cell (1, 2): X = 1.0, O = 0.0
Cell (1, 3): X = 1.0, O = 0.0
Cell (2, 1): X = 0.0, O = 1.0
Cell (2, 2): X = 0.0, O = 1.0
Cell 